In [30]:
from scipy.stats import multivariate_normal
from massart import addNoise, ground_truth_permutations, flip_ranking, massart_noisy_ranking
from LabelRankers import *
import pickle

In [110]:
import pandas as pd
from sklearn.model_selection import train_test_split

dataset = pd.read_csv('LR_DATASETS/elevators.txt')

X = dataset.drop(columns=dataset.loc[:, 'ranking':].columns).to_numpy()

P = dataset.drop(columns=dataset.loc[:, :'ranking'].columns).to_numpy().astype('int')

X_train, X_test, P_train, P_test = train_test_split(X, P, test_size=0.2)

In [111]:
d = X.shape[1]
k = P.shape[1]

##### Train models using data with RCN

In [112]:
LR_DT_R = {} # Decision Tree Regression
LR_DT_C = {} # Decision Tree Classification
LR_RF_R = {} # Random Forest Regression
LR_RF_C = {} # Random Forest Classification
LR_HH_C = {} # Homogeneous Halfspace Classification

LR_DT_R = LabelwiseDecisionTreeLR().fit(X_train, P_train)
LR_DT_C = PairwiseDecisionTreeLR().fit(X_train, P_train)
LR_RF_R = LabelwiseRandomForestLR().fit(X_train, P_train)
LR_RF_C = PairwiseRandomForestLR().fit(X_train, P_train)
LR_HH_C = PairwiseHalfspaceLR(0.005, 0.1, 0.8).fit(X_train, P_train)

##### Predict rankings for all classifiers and etas

In [113]:
LR_DT_R_preds = LR_DT_R.predict(X_test)
LR_DT_C_preds = LR_DT_C.predict(X_test)
LR_RF_R_preds = LR_RF_R.predict(X_test)
LR_RF_C_preds = LR_RF_C.predict(X_test)
LR_HH_C_preds = LR_HH_C.predict(X_test)

##### Find scores for all classifiers and etas

In [114]:
LR_DT_R_KT_corr = mean_kendall_rank_corr(P_test, LR_DT_R_preds)
LR_DT_C_KT_corr = mean_kendall_rank_corr(P_test, LR_DT_C_preds)
LR_RF_R_KT_corr = mean_kendall_rank_corr(P_test, LR_RF_R_preds)
LR_RF_C_KT_corr = mean_kendall_rank_corr(P_test, LR_RF_C_preds)
LR_HH_C_KT_corr = mean_kendall_rank_corr(P_test, LR_HH_C_preds)

In [115]:
print(LR_DT_R_KT_corr)
print(LR_DT_C_KT_corr)
print(LR_RF_R_KT_corr)
print(LR_RF_C_KT_corr)
print(LR_HH_C_KT_corr)

0.6812751004016063
0.6929886211512719
0.784119812583668
0.8071285140562249
0.5867469879518072
